In [20]:
from dataclasses import dataclass
from typing import Callable
from langchain.agents.middleware.types import ModelRequest, ModelResponse, before_agent, wrap_model_call
from langchain.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime
from langgraph.store.memory import InMemoryStore


from dotenv import load_dotenv
import os

load_dotenv()

@dataclass
class Context:
    user_id: str



@tool
def create_file(file_path: str, file_content: str) -> str:
    """It will add the provided file to the memory"""
    
    try:
        with open(f"{file_path}", "w") as f:
            f.write(file_content)
        return f"Successfully added {file_content[:100]} to the {file_path}"
    except Exception as e:
        return f"Error {e}"

@tool
def get_list_syllabus() -> list[str]:
    """It will return the list of all the saved syllabuses"""

    try:
        return os.listdir("memory/syllabuses/")
    except Exception as e:
        return f"Error {e}"

@tool
def read_file(file_path: str) -> str:
    """It will return the content of the provided file"""
    try:
        with open(file_path, "r") as f:
            return f.read()
    except Exception as e:
        return f"Error {e}"


# # Middleware
# @wrap_model_call
# def install_user_preferences(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]):
#     """This middleware will install the user preferences"""
#     print(request)
#     user_id = request.runtime.context.user_id
#     if user_id:
#         # get user preferences from the database
#         # user_pref = get_user_profile(user_id, db_connection)
#         try:
#             with open(f"memory/user_profile.md", "r") as f:
#                 user_profile = "This is the user profile:\n\n" + f.read()
#         except Exception as e:
#             user_profile = ""

#         messages = [HumanMessage(content=user_profile),
#                     *request.messages]
#         request = request.override(messages=messages)
#     return handler(request)


# @wrap_model_call
# def install_skills(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]):
#     """This middleware will install the skills' names and descriptions"""
    
#     try:
#         with open("skills/syllabi/SKILL.md", "r") as f:
#             skills = f.read()
#     except Exception as e:
#         skills = ""

#     messages = [HumanMessage(content=skills),
#                 *request.messages]
#     request = request.override(messages=messages)
#     return handler(request)


tools = [get_list_syllabus, read_file, create_file]

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


model = ChatOpenAI(
    model="gpt-4o-mini", 
    api_key=OPENAI_API_KEY
)

In [21]:
import platform
from pathlib import Path
from nanobot_code.memory import MemoryStore
from nanobot_code.skills import SkillsLoader

class ContextBuilder:
    """
    Builds the context (system prompt + messages) for the agent.
    
    Assembles bootstrap files, memory, skills, and conversation history
    into a coherent prompt for the LLM.
    """
    
    BOOTSTRAP_FILES = ["AGENTS.md", "SOUL.md", "USER.md", "TOOLS.md", "IDENTITY.md"]
    
    def __init__(self, workspace: Path):
        self.workspace = workspace
        self.memory = MemoryStore(workspace)
        self.skills = SkillsLoader(workspace)
    def build_system_prompt(self, skill_names: list[str] | None = None) -> str:
        """
        Build the system prompt from bootstrap files, memory, and skills.
        
        Args:
            skill_names: Optional list of skills to include.
        
        Returns:
            Complete system prompt.
        """
        parts = []
        
        # Core identity
        parts.append(self._get_identity())
        
        # Bootstrap files
        bootstrap = self._load_bootstrap_files()
        if bootstrap:
            parts.append(bootstrap)
        
        # Memory context
        memory = self.memory.get_memory_context()
        if memory:
            parts.append(f"# Memory\n\n{memory}")
        
        # Skills - progressive loading
        # 1. Always-loaded skills: include full content
        always_skills = self.skills.get_always_skills()
        if always_skills:
            always_content = self.skills.load_skills_for_context(always_skills)
            if always_content:
                parts.append(f"# Active Skills\n\n{always_content}")
        
        # 2. Available skills: only show summary (agent uses read_file to load)
        skills_summary = self.skills.build_skills_summary()
        if skills_summary:
            parts.append(f"""# Skills

    The following skills extend your capabilities. To use a skill, read its SKILL.md file using the read_file tool.
    Skills with available="false" need dependencies installed first - you can try installing them with apt/brew.

    {skills_summary}""")
        
        return "\n\n---\n\n".join(parts)

    def _get_identity(self) -> str:
        """Get the core identity section."""
        from datetime import datetime
        import time as _time
        now = datetime.now().strftime("%Y-%m-%d %H:%M (%A)")
        tz = _time.strftime("%Z") or "UTC"
        workspace_path = str(self.workspace.expanduser().resolve())
        system = platform.system()
        runtime = f"{'macOS' if system == 'Darwin' else system} {platform.machine()}, Python {platform.python_version()}"
        
        return f"""# nanobot 🐈

    You are nanobot, a helpful AI assistant. 

    ## Current Time
    {now} ({tz})

    ## Runtime
    {runtime}

    ## Workspace
    Your workspace is at: {workspace_path}
    - Long-term memory: {workspace_path}/memory/MEMORY.md
    - History log: {workspace_path}/memory/HISTORY.md (grep-searchable)
    - Custom skills: {workspace_path}/skills/{{skill-name}}/SKILL.md

    Reply directly with text for conversations. Only use the 'message' tool to send to a specific chat channel.

    ## Tool Call Guidelines
    - Before calling tools, you may briefly state your intent (e.g. "Let me check that"), but NEVER predict or describe the expected result before receiving it.
    - Before modifying a file, read it first to confirm its current content.
    - Do not assume a file or directory exists — use list_dir or read_file to verify.
    - After writing or editing a file, re-read it if accuracy matters.
    - If a tool call fails, analyze the error before retrying with a different approach.

    ## Memory
    - Remember important facts: write to {workspace_path}/memory/MEMORY.md
    - Recall past events: grep {workspace_path}/memory/HISTORY.md"""

    def _load_bootstrap_files(self) -> str:
        """Load all bootstrap files from workspace."""
        parts = []
        
        for filename in self.BOOTSTRAP_FILES:
            file_path = self.workspace / filename
            if file_path.exists():
                content = file_path.read_text(encoding="utf-8")
                parts.append(f"## {filename}\n\n{content}")
        
        return "\n\n".join(parts) if parts else ""


context_builder = ContextBuilder(Path("~/.nanobot/workspace"))

system_prompt = context_builder.build_system_prompt()
print(system_prompt)

# nanobot 🐈

    You are nanobot, a helpful AI assistant. 

    ## Current Time
    2026-02-25 10:49 (Wednesday) (+05)

    ## Runtime
    macOS arm64, Python 3.13.5

    ## Workspace
    Your workspace is at: /Users/nurma/.nanobot/workspace
    - Long-term memory: /Users/nurma/.nanobot/workspace/memory/MEMORY.md
    - History log: /Users/nurma/.nanobot/workspace/memory/HISTORY.md (grep-searchable)
    - Custom skills: /Users/nurma/.nanobot/workspace/skills/{skill-name}/SKILL.md

    Reply directly with text for conversations. Only use the 'message' tool to send to a specific chat channel.

    ## Tool Call Guidelines
    - Before calling tools, you may briefly state your intent (e.g. "Let me check that"), but NEVER predict or describe the expected result before receiving it.
    - Before modifying a file, read it first to confirm its current content.
    - Do not assume a file or directory exists — use list_dir or read_file to verify.
    - After writing or editing a file, re-read it 

In [22]:
# with open("temp_system_prompt.txt", "w") as f:
#     f.write(system_prompt)

In [23]:
agent = create_agent(
    model=model, 
    tools=tools,
    system_prompt=system_prompt,
    # middleware=[install_user_preferences],
    checkpointer=InMemorySaver(),
    store=InMemoryStore(),
    context_schema=Context
)


In [24]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "5"}}

messages = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "q"]:
        print("Goodbye!")
        break
    messages.append(HumanMessage(content=user_input))

    messages[-1].pretty_print()

    result = agent.invoke(
        {"messages": messages},
        config=config,
        context=Context(user_id="customer_123")
    )
    
    messages= result["messages"]

    result["messages"][-1].pretty_print()

================================ Human Message =================================

hey
================================== Ai Message ==================================

Hello! How can I assist you today?
================================ Human Message =================================

what did we talk about?
================================== Ai Message ==================================

Here's a brief summary of our recent conversations:

- On **February 24**, you finished summarizing notes from Module UNIX videos.
- On the same day, you set a daily reminder preference to remind you every day at 06:00 (UTC+5) with a short bullet list of tasks for the day, generated from the conversation history.
- On **February 19**, we discussed your tasks for the day, which included finishing the LangChain documentation on "Messages," learning about the "nanobot" memory system, and researching service as a business and freelance strategies.

If you need more details or have specific questions about 

In [25]:
messages

[HumanMessage(content='hey', additional_kwargs={}, response_metadata={}, id='c8bb2c78-4822-49a4-9e53-17ad60ea1f03'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 573, 'total_tokens': 583, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DD23ofMVmzXdJxffZIfwzy17nv7LJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9358-2d09-7c52-9ec2-6e4f076a50ef-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 573, 'output_tokens': 10, 'total_tokens': 583, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [28]:
messages[3].tool_calls

[{'name': 'read_file',
  'args': {'file_path': '/Users/nurma/.nanobot/workspace/memory/HISTORY.md'},
  'id': 'call_g2PL1kEtfvhTJR3t0hFcHIXk',
  'type': 'tool_call'}]

In [19]:
import re


def load_skill(name: str) -> str:
    with open(f"skills/{name}/SKILL.md", "r") as f:
        return f.read()

def get_skill_metadata(name: str) -> dict | None:
    """
    Get metadata from a skill's frontmatter.
    
    Args:
        name: Skill name.
    
    Returns:
        Metadata dict or None.
    """
    content = load_skill(name)
    if not content:
        return None
    
    if content.startswith("---"):
        match = re.match(r"^---\n(.*?)\n---", content, re.DOTALL)
        if match:
            # Simple YAML parsing
            metadata = {}
            for line in match.group(1).split("\n"):
                if ":" in line:
                    key, value = line.split(":", 1)
                    metadata[key.strip()] = value.strip().strip('"\'')
            return metadata
    
    return None

get_skill_metadata("syllabus")

{'name': 'syllabi_manager',
 'description': 'Use this skill everytime when you need access to the actual information in the syllabus, or if you need to add a new syllabus',
 'allowed-tools': 'get_syllabus_content, get_list_syllabus, add_syllabus'}

In [ ]:
@tool
def read_file(file_path: str) -> str:
    """It will return the content of the provided file"""
    with open(file_path, "r") as f:
        return f.read()

read_file("/Users/nurma/vscode_projects/nanobot/messages.json")

In [13]:
from pathlib import Path
from skills import SkillsLoader

skills = SkillsLoader(Path("."), Path("skills"))

skill_list = skills.list_skills()
skill_list


[{'name': 'syllabus',
  'path': 'skills/syllabus/SKILL.md',
  'source': 'workspace'}]

In [11]:
print(skills.build_skills_summary())

<skills>
  <skill available="true">
    <name>syllabus</name>
    <description>Use this skill everytime when you need access to the actual information in the syllabus, or if you need to add a new syllabus</description>
    <location>skills/syllabus/SKILL.md</location>
  </skill>
</skills>


In [18]:
print(skills.load_skill("syllabus"))


---
name: syllabi_manager
description: Use this skill everytime when you need access to the actual information in the syllabus, or if you need to add a new syllabus
allowed-tools: get_syllabus_content, get_list_syllabus, add_syllabus
---

First you must fetch all the accessible syllabi using the tool get_list_syllabus

Then, you can get the content of the chosen syllabus by using the tool get_syllabus_content

add_syllabus - use this tool to add a new syllabus if a new one is provided.


In [21]:
print(skills.load_skills_for_context(["syllabus"]))

### Skill: syllabus

First you must fetch all the accessible syllabi using the tool get_list_syllabus

Then, you can get the content of the chosen syllabus by using the tool get_syllabus_content

add_syllabus - use this tool to add a new syllabus if a new one is provided.


In [22]:
skills.get_skill_metadata("syllabus")

{'name': 'syllabi_manager',
 'description': 'Use this skill everytime when you need access to the actual information in the syllabus, or if you need to add a new syllabus',
 'allowed-tools': 'get_syllabus_content, get_list_syllabus, add_syllabus'}

In [13]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}

messages = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "q"]:
        print("Goodbye!")
        break
    messages.append(HumanMessage(content=user_input))

    messages[-1].pretty_print()

    result = agent.invoke(
        {"messages": messages},
        config=config,
        context=Context(user_id="customer_123")
    )
    
    messages.append(result["messages"][-1])

    result["messages"][-1].pretty_print()


================================ Human Message =================================

hello
ModelRequest(model=ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x114b26c40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x114b27e10>, root_client=<openai.OpenAI object at 0x114b25350>, root_async_client=<openai.AsyncOpenAI object at 0x114b26d70>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), messages=[HumanMessage(conten

In [8]:
result["messages"][2].tool_calls

[{'name': 'add_syllabus',
  'args': {'syllabus_name': 'История Казахстана',
   'syllabus_content': 'Максимальное количество пропусков по курсу: 3 занятия.'},
  'id': 'call_9S5AnCFRYDqOruCpI5eUifNK',
  'type': 'tool_call'},
 {'name': 'add_syllabus',
  'args': {'syllabus_name': 'Линейная алгебра',
   'syllabus_content': 'Предпочтения: Люблю линейную алгебру.'},
  'id': 'call_lC0b7F0Eiy22c5IMTYhPhQzY',
  'type': 'tool_call'}]